In [27]:
import pandas as pd
df = pd.read_csv('../../data/Leases.csv')
print(df.info())
print(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194685 entries, 0 to 194684
Data columns (total 35 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   year                            194685 non-null  int64  
 1   quarter                         194685 non-null  object 
 2   monthsigned                     193822 non-null  float64
 3   market                          194685 non-null  object 
 4   building_name                   157999 non-null  object 
 5   building_id                     194685 non-null  object 
 6   address                         194685 non-null  object 
 7   region                          194685 non-null  object 
 8   city                            194685 non-null  object 
 9   state                           194685 non-null  object 
 10  zip                             194684 non-null  float64
 11  internal_submarket              194685 non-null  object 
 12  internal_class  

In [ ]:
jkdf.groupby('internal_submarket')

In [2]:
import folium
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Load the data
leases_df = pd.read_csv("../../data/Leases.csv")
leases_df["full_address"] = leases_df["address"] + ", " + leases_df["city"] + ", " + leases_df["state"]
print(leases_df.full_address)


0                      10 Glenlake Pky NE, Atlanta, GA
1                    3330 Cumberland Blvd, Atlanta, GA
2                    1000 Parkwood Cir SE, Atlanta, GA
3                   1100 Circle 75 Pky SE, Atlanta, GA
4                        1200 Ashwood Pky, Atlanta, GA
                              ...                     
194680    3111 W Dr Martin Luther King Blvd, Tampa, FL
194681                         1 Midtown Pl, Tampa, FL
194682                       400 N Tampa St, Tampa, FL
194683                      400 N Ashley Dr, Tampa, FL
194684    3101 W Dr Martin Luther King Blvd, Tampa, FL
Name: full_address, Length: 194685, dtype: object


In [13]:
leases_df['test'] = leases_df['internal_submarket'] + ", " + leases_df['city'] + ", " + leases_df['state']

In [17]:
leases_df['test'].unique()

array(['Central Perimeter, Atlanta, GA', 'Northwest, Atlanta, GA',
       'Northeast, Peachtree Corners, GA', ...,
       'Southeast, American Fork, UT', 'Northend, Stanwood, WA',
       'Lower Bucks County, Fairless Hills, PA'],
      shape=(1411,), dtype=object)

In [18]:
unique_addresses = pd.DataFrame({'address': leases_df['test'].unique()})

In [22]:
unique_addresses.replace('Central Perimeter', 'Perimeter Center')

,address
0,"Central Perimeter, Atlanta, GA"
1,"Northwest, Atlanta, GA"
2,"Northeast, Peachtree Corners, GA"
3,"Midtown, Atlanta, GA"
4,"Northeast, Gainesville, GA"
...,...
1406,"Route 495 North East, Middleton, MA"
1407,"Urban Essex, NEWARK, NJ"
1408,"Southeast, American Fork, UT"
1409,"Northend, Stanwood, WA"


In [23]:
from tqdm import tqdm

In [26]:
# Geocode
import numpy as np
geolocator = Nominatim(user_agent="rent_mapper")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=3)

def geocode_apply_func(address):
    try:
        res = geocode(address)
    except:
        res = np.nan
    return res

for lease_loc in unique_addresses.address:
    print(lease_loc)
    res = geocode('Atlanta, GA')
    print(res)
    print(res.latitude)
    print(res.longitude)
    break
tqdm.pandas(desc='apply bar')
unique_addresses['location'] = unique_addresses['address'].progress_apply(geocode_apply_func)

# leases_df["location"] = leases_df["full_address"].apply(geocode)
# leases_df["lat"] = leases_df["location"].apply(lambda loc: loc.latitude if loc else None)
# leases_df["lon"] = leases_df["location"].apply(lambda loc: loc.longitude if loc else None)

# Filter valid points
# leases_geo = leases_df.dropna(subset=["lat", "lon", "overall_rent"])

Central Perimeter, Atlanta, GA
Atlanta, Fulton County, Georgia, United States
33.7489924
-84.3902644


apply bar:   5%|▍         | 65/1411 [03:12<1:07:19,  3.00s/it]RateLimiter caught an error, retrying (0/2 tries). Called with (*('Route 128 West, Lexington, MA',), **{}).
Traceback (most recent call last):
  File "/Users/aashishcheruvu/miniconda3/envs/datafest/lib/python3.13/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
  File "/Users/aashishcheruvu/miniconda3/envs/datafest/lib/python3.13/site-packages/urllib3/connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
  File "/Users/aashishcheruvu/miniconda3/envs/datafest/lib/python3.13/http/client.py", line 1430, in getresponse
    response.begin()
    ~~~~~~~~~~~~~~^^
  File "/Users/aashishcheruvu/miniconda3/envs/datafest/lib/python3.13/http/client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ~~~~~~~~~~~~~~~~~^^
  File "/Users/aashishcheruvu/miniconda3/envs/datafest/lib/python3.13/http/client.p

In [ ]:
# Create Folium Map
m = folium.Map(location=[leases_geo["lat"].mean(), leases_geo["lon"].mean()], zoom_start=6)

for _, row in leases_geo.iterrows():
    popup_text = f"{row['building_name']}<br>Rent: ${row['overall_rent']:.2f}/sqft"
    folium.CircleMarker(
        location=(row["lat"], row["lon"]),
        radius=5,
        color='blue',
        fill=True,
        fill_opacity=0.6,
        popup=popup_text
    ).add_to(m)

# Save map
m.save("lease_rent_map.html")